<a href="https://colab.research.google.com/github/ayulockin/SwAV-TF/blob/master/Fully_Supervised_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initial setup

In [1]:
%%capture
!pip install wandb

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras import models

import matplotlib.pyplot as plt 
import numpy as np
import random
import time
import os

tf.random.set_seed(666)
np.random.seed(666)

tfds.disable_progress_bar()

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

## Dataset gathering and preparation

In [10]:
# Gather Flowers dataset
train_ds, validation_ds = tfds.load(
    "tf_flowers",
    split=["train[:85%]", "train[85%:]"],
    as_supervised=True
)

AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

@tf.function
def scale_resize_image(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, (224, 224)) # Resizing to highest resolution used while training swav
    return (image, label)

training_ds = (
    train_ds
    .map(scale_resize_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

testing_ds = (
    validation_ds
    .map(scale_resize_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

## ResNet50 from scratch

In [11]:
def get_training_model():
    inputs = layers.Input(shape=(224, 224, 3))
    EXTRACTOR = tf.keras.applications.ResNet50(weights=None, include_top=False,
        input_shape=(224, 224, 3))
    x = EXTRACTOR(inputs, training=True)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(5, activation="softmax")(x)
    classifier = models.Model(inputs=inputs, outputs=x)
    
    return classifier

In [12]:
model = get_training_model()
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 23,544,837
Non-trainable params: 53,120
_________________________________________________________________


## Callback

In [13]:
# Early Stopping to prevent overfitting
early_stopper = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, verbose=2, restore_best_weights=True)

## Without Augmentation

#### Training

In [14]:
# get model and compile
tf.keras.backend.clear_session()
model = get_training_model()
model.summary()

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer="adam")

# initialize wandb run
wandb.init(entity='authors', project='swav-tf', id='resnet50-scratch')

# train 
history = model.fit(training_ds,
                 validation_data=(testing_ds),
                 epochs=100,
                 callbacks=[WandbCallback(),
                            early_stopper])

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 23,544,837
Non-trainable params: 53,120
_________________________________________________________________


Epoch 1/100
25/25 [==============================] - 19s 740ms/step - loss: 2.1911 - acc: 0.4321 - val_loss: 1.4640 - val_acc: 0.5091
Epoch 2/100
25/25 [==============================] - 17s 664ms/step - loss: 1.1474 - acc: 0.5497 - val_loss: 1.1207 - val_acc: 0.5982
Epoch 3/100
25/25 [==============================] - 17s 677ms/step - loss: 0.9998 - acc: 0.6321 - val_loss: 0.9639 - val_acc: 0.6509
Epoch 4/100
25/25 [==============================] - 17s 673ms/step - loss: 0.8823 - acc: 0.6686 - val_loss: 0.9426 - val_acc: 0.6600
Epoch 5/100
25/25 [==============================] - 17s 668ms/step - loss: 0.8131 - acc: 0.7042 - val_loss: 0.9311 - val_acc: 0.6364
Epoch 6/100
25/25 [==============================] - 16s 621ms/step - loss: 0.7122 - acc: 0.7397 - val_loss: 0.9827 - val_acc: 0.6582
Epoch 7/100
25/25 [==============================] - 16s 623ms/step - loss: 0.6778 - acc: 0.7513 - val_loss: 1.0428 - val_acc: 0.6509
Epoch 00007: early stopping


#### Evaluation

In [15]:
loss, acc = model.evaluate(testing_ds)
wandb.log({'Test Accuracy': round(acc*100, 2)})

5/5 [==============================] - 1s 177ms/step - loss: 0.9311 - acc: 0.6364


# Training with Augmentation


#### Augmentation

In [16]:
# Configs
CROP_SIZE = 224
MIN_SCALE = 0.5
MAX_SCALE = 1.

# Experimental options
options = tf.data.Options()
options.experimental_optimization.noop_elimination = True
options.experimental_optimization.map_vectorization.enabled = True
options.experimental_optimization.apply_default_optimizations = True
options.experimental_deterministic = False
options.experimental_threading.max_intra_op_parallelism = 1

In [17]:
@tf.function
def scale_image(image, label):
	image = tf.image.convert_image_dtype(image, tf.float32)
	return (image, label)

@tf.function
def random_apply(func, x, p):
	return tf.cond(
		tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
				tf.cast(p, tf.float32)),
		lambda: func(x),
		lambda: x)
 
@tf.function
def random_resize_crop(image, label):
  # Conditional resizing
  image = tf.image.resize(image, (260, 260))
  # Get the crop size for given min and max scale
  size = tf.random.uniform(shape=(1,), minval=MIN_SCALE*260,
		          maxval=MAX_SCALE*260, dtype=tf.float32)
  size = tf.cast(size, tf.int32)[0]
  # Get the crop from the image
  crop = tf.image.random_crop(image, (size,size,3))
  crop_resize = tf.image.resize(crop, (CROP_SIZE, CROP_SIZE))
  
  return crop_resize, label

@tf.function
def tie_together(image, label):
  # Scale the pixel values
  image, label = scale_image(image , label)
  # random horizontal flip
  image = random_apply(tf.image.random_flip_left_right, image, p=0.5)
  # Random resized crops
  image, label = random_resize_crop(image, label)
  
  return image, label

In [18]:
trainloader = (
	train_ds
	.shuffle(1024)
	.map(tie_together, num_parallel_calls=AUTO)
	.batch(BATCH_SIZE)
	.prefetch(AUTO)
)

trainloader = trainloader.with_options(options)

#### Training

In [19]:
# get model and compile
tf.keras.backend.clear_session()
model = get_training_model()
model.summary()

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer="adam")

# initialize wandb run
wandb.init(entity='authors', project='swav-tf', id='resnet50-scratch-aug')

# train 
history = model.fit(training_ds,
                 validation_data=(testing_ds),
                 epochs=100,
                 callbacks=[WandbCallback(),
                            early_stopper])

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 23,544,837
Non-trainable params: 53,120
_________________________________________________________________


Epoch 1/100
25/25 [==============================] - 18s 706ms/step - loss: 2.2764 - acc: 0.4083 - val_loss: 1.4937 - val_acc: 0.5527
Epoch 2/100
25/25 [==============================] - 17s 683ms/step - loss: 1.2415 - acc: 0.5865 - val_loss: 1.2379 - val_acc: 0.6200
Epoch 3/100
25/25 [==============================] - 17s 675ms/step - loss: 1.0328 - acc: 0.6314 - val_loss: 0.9624 - val_acc: 0.6582
Epoch 4/100
25/25 [==============================] - 16s 622ms/step - loss: 0.9649 - acc: 0.6615 - val_loss: 0.9770 - val_acc: 0.6436
Epoch 5/100
25/25 [==============================] - 16s 627ms/step - loss: 0.8654 - acc: 0.6936 - val_loss: 1.0189 - val_acc: 0.6418
Epoch 00005: early stopping


#### Evaluation

In [20]:
loss, acc = model.evaluate(testing_ds)
wandb.log({'Test Accuracy': round(acc*100, 2)})

5/5 [==============================] - 1s 194ms/step - loss: 0.9624 - acc: 0.6582
